<a href="https://colab.research.google.com/github/divyaurquijo/PYRAT/blob/main/Episode5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b><center>How to use this file with PyRat?</center></b></h1>

Google Colab provides an efficient environment for writing codes collaboratively with your group. For us, teachers, it allows to come and see your advancement from time to time, and help you solve some bugs if needed.

The PyRat software is a complex environment that takes as an input an AI file (as this file). It requires some resources as well as a few Python libraries, so we have installed it on a virtual machine for you.

PyRat is a local program, and Google Colab is a distant tool. Therefore, we need to indicate the PyRat software where to get your code! In order to submit your program to PyRat, you should follow the following steps:

1.   In this notebook, click on *Share* (top right corner of the navigator). Then, change sharing method to *Anyone with the link*, and copy the sharing link;
2.   On the machine where the PyRat software is installed, start a terminal and navigate to your PyRat directory;
3.   Run the command `python ./pyrat.py --rat "<link>" <options>`, where `<link>` is the share link copied in step 1. (put it between quotes), and `<options>` are other PyRat options you may need.

<h1><b><center>Pre-defined constants</center></b></h1>

A PyRat program should -- at each turn -- decide in which direction to move. This is done in the `turn` function later in this document, which should return one of the following constants:

In [ ]:
MOVE_DOWN = 'D'
MOVE_LEFT = 'L'
MOVE_RIGHT = 'R'
MOVE_UP = 'U'

<h1><b><center>Your coding area</center></b></h1>

Please put your functions, imports, constants, etc. between this text and the PyRat functions below. You can add as many code cells as you want, we just ask that you keep things organized (one function per cell, commented, etc.), so that it is easier for the teachers to help you debug your code!

In [ ]:
import heapq
import math

def priority_queue () :
    return []

def is_empty (queue) :
    return queue == priority_queue()

def add_or_replace (queue, key, value) :
    heapq.heappush(queue, (key, value))

In [ ]:
def dijkstra(graph, start_vertex, target_location):
    # initialize
    min_heap = priority_queue()
    add_or_replace(min_heap, 0, start_vertex)
    visited_location = []
    shortest_distances = {start_vertex: 0}
    parent = {}

    # algorithm loop
    while not(is_empty(min_heap)):
        distance, v = heapq.heappop(min_heap)
        if v not in visited_location:
            visited_location.append(v)
            if v  == target_location:
                break
            for neighbor in graph[v]:
                distance_through_v = distance + graph[v][neighbor]
                distance_minimale = shortest_distances.get(neighbor)
                if (distance_minimale is None or distance_through_v < distance_minimale):
                    shortest_distances[neighbor] = distance_through_v
                    parent[neighbor] = v
                    add_or_replace(min_heap, distance_through_v, neighbor)
    distance = shortest_distances.get(target_location)
    locations = []
    if distance is not None:
        vertex = target_location
        while vertex is not None:
            locations = [vertex] + locations
            vertex = parent.get(vertex)
    return locations, distance


In [ ]:
def build_meta_graph (graph, locations, start_vertex) :
    # Return the meta-graph and all necessary routing tables
    complete_graph = {}              # On crée un dictionnaire pour contenir le méta-graph
    locations.append(start_vertex)
    for vertex in locations:      # On regarde pour chaque position de fromages + player_location
        complete_graph[vertex] = {}      # On crée une clé pour chaque locations
        for neighbor in locations:
            if vertex != neighbor:
                complete_graph[vertex][neighbor] = dijkstra(graph, vertex, neighbor)      # on ajoute la distance grâce à dijkstra
    return complete_graph

In [ ]:
def unvisited_neighbors(graph, current_vertex, visited_neighbors):
    neighbors_to_see = []
    for vertex in graph[current_vertex]:
        if vertex not in visited_neighbors:
          neighbors_to_see.append(vertex)
    return neighbors_to_see

In [ ]:
def tsp(meta_graph, start_vertex) :
    shortest_distance = math.inf
    best_path = []
    def _tsp(current_vertex, current_length, neighbors_to_see, routing_table) :
        # Recursive implementation of the tree exploration
        nonlocal shortest_distance
        nonlocal best_path
        if neighbors_to_see == []:
            if current_length < shortest_distance:
                shortest_distance = current_length
                best_path = routing_table
        else:
            for neighbor in neighbors_to_see:
                new_length = current_length + meta_graph[current_vertex][neighbor][1]
                if new_length < shortest_distance:
                    _tsp(neighbor, new_length, unvisited_neighbors(meta_graph, neighbor, routing_table), routing_table + [neighbor])

  # Initial call
    _tsp(start_vertex, 0, unvisited_neighbors(meta_graph, start_vertex, []), [start_vertex])
    return shortest_distance , best_path

In [ ]:
def meta_graph_route_to_route (meta_graph, best_path, start_vertex) :
    locations = [start_vertex]
    for i in range(len(best_path)-1):
        cheese_1 = best_path[i]
        cheese_2 = best_path[i+1]
        route = meta_graph[cheese_1][cheese_2][0]
        route.pop(0)
        locations += route
    return locations

In [ ]:
def moves_from_locations(locations) :
    # Transform a series of locations into corresponding moves to realize it
    n = len(locations)
    moves = []
    for i in range(n-1):
        difference = (locations[i+1][0]-locations[i][0],locations[i+1][1]-locations[i][1])
        if difference == (0, -1) :
            moves.append(MOVE_DOWN)
        elif difference == (0, 1) :
            moves.append(MOVE_UP)
        elif difference == (1, 0) :
            moves.append(MOVE_RIGHT)
        elif difference == (-1, 0) :
            moves.append(MOVE_LEFT)
        else :
            raise Exception("Impossible move")
    return moves

<h1><b><center>PyRat functions</center></b></h1>

The `preprocessing` function is called at the very start of a game. It is attributed a longer time compared to `turn`, which allows you to perform intensive computations. If you store the results of these computations into **global** variables, you will be able to reuse them in the `turn` function.

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze where the game takes place. This structure associates each cell with the dictionry of its neighbors. In that dictionary of neighbors, keys are cell coordinates, and associated values the number of moves required to reach that neighbor. As an example, `list(maze_map[(0, 0)].keys())` returns the list of accessible cells from `(0, 0)`. Then, if for example `(0, 1)` belongs to that list, one can access the number of moves to go from `(0, 0)` to `(0, 1)` by the code `maze_map[(0, 0)][0, 1)]`.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The initial location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The initial location of your opponent's character in the maze.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The initial location of all pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take for preprocessing before the game starts checking for moves.

*Output:*
*   This function does not output anything.

In [ ]:
def preprocessing (maze_map, maze_width, maze_height, player_location, opponent_location, pieces_of_cheese, time_allowed) :
   
    meta_graph = build_meta_graph (maze_map, pieces_of_cheese, player_location)
    shortest_distance, best_path = tsp(meta_graph, player_location)
    locations = meta_graph_route_to_route(meta_graph, best_path, player_location)
    global chemin
    chemin = moves_from_locations(locations)



    # Example prints that appear in the shell only at the beginning of the game
    # Remove them when you write your own program
    print("maze_map", type(maze_map), maze_map)
    print("maze_width", type(maze_width), maze_width)
    print("maze_height", type(maze_height), maze_height)
    print("player_location", type(player_location), player_location)
    print("opponent_location", type(opponent_location), opponent_location)
    print("pieces_of_cheese", type(pieces_of_cheese), pieces_of_cheese)
    print("time_allowed", type(time_allowed), time_allowed)

The `turn` function is called each time the game is waiting
for the player to make a decision (*i.e.*, to return a move among those defined above).

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze. It is the same as in the `preprocessing` function, just given here again for convenience.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The current location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The current location of your opponent's character in the maze.
*   `player_score` -- **float** -- Your current score.
*   `opponent_score` -- **float** -- The opponent's current score.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The location of remaining pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take to return a move to apply before another time starts automatically.

*Output:*
*   A chosen move among `MOVE_UP`, `MOVE_DOWN`, `MOVE_LEFT` or `MOVE_RIGHT`.

In [ ]:
def turn (maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed) :
    
    # We go up at every turn
    # You should replace this with more intelligent decisions
    
    return chemin.pop(0)